In [40]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [45]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
#     add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
#                          'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
#                         }
    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['submission in progress', 'current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
kaufman_sets = [
'c85de504-96cf-4856-b7f3-398e6bf1e37b',
'c50006d4-a98d-4366-bd5b-650fee1696ee',
'43586c61-a527-43d3-b95a-1bc063fa1fb7',
'e2b336fb-dda9-4eff-9125-0603cc99916e',
'f41f9b4b-9f13-4e25-abed-13fcf99b5a73',
'1ed849f5-cf49-4190-84a9-161f13906c21',
'c33e4a8d-cd5e-4bf2-8973-88a46efd2173',
'd5fd5f93-8ffc-4bdc-8db9-1457d119248b',
]
sprite_sets = ['cc82a2af-95e7-4447-b571-87d4785b33ec']

track_loop_sets = [
'9b1063f5-c891-44a8-b44a-3b4879257366',
'c5db8085-92bd-4d06-a22c-7958e4a78e19',]

dekker_mc = [
'51d9ac15-e55e-4f64-85e5-01998e349cf1',
'998573fb-384e-4792-a621-254d705d4ec9',
]

my_exp = dekker_mc[1]

print my_exp
store_dict = {} 
item_uuids = []
store, uuids = record_object(my_exp, store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




998573fb-384e-4792-a621-254d705d4ec9
94
94


In [46]:
## This part is only for making non file objects "in submission"
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        for raw_data in store[a_type]:
            print '    SKIPPING FILES', a_type, raw_data['status'], raw_data['uuid']
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print '    DELETED DATA', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['submission in progress', 'current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released"}
            #ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

quality_metric_fastqc in review by lab 177c3121-c621-43a7-b00b-c5c1f7406135
quality_metric_fastqc in review by lab 95ab97cd-404a-4e86-8f07-e4ac5469c159
quality_metric_fastqc in review by lab 1f30ebe0-4e50-4599-b3da-d313868a9980
quality_metric_fastqc in review by lab 1ef636af-7830-4a85-afb8-1062aa136116
quality_metric_fastqc in review by lab 13a3afc2-e3c2-4af4-88fb-58d96d8a5686
quality_metric_fastqc in review by lab 45f9f32b-fb0b-4449-88b9-1a2771eb40ce
quality_metric_fastqc in review by lab 6071137f-3a0e-42c4-9dad-7f3ab445e4aa
quality_metric_fastqc in review by lab bd098a73-bccc-4cd1-8a87-81344557bc2d
quality_metric_fastqc in review by lab 421f776c-e515-4217-81f7-18ca28b527a3
quality_metric_fastqc in review by lab 0a9e5ad5-20d6-4c0d-87dc-937c43c4991f
quality_metric_fastqc in review by lab 3f778ec7-e5ea-44ed-9686-cd2e53e5bb71
quality_metric_fastqc in review by lab d4d84ff7-43bc-4419-94a3-fac27e86103d
quality_metric_fastqc in review by lab 20fbfa5a-209e-4ce6-bb6c-02f7b0e4237d
quality_metr

In [47]:
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            continue        
        if raw_data['status'] not in ['submission in progress', 'current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "submission in progress"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

quality_metric_fastqc in review by lab 177c3121-c621-43a7-b00b-c5c1f7406135
quality_metric_fastqc in review by lab 95ab97cd-404a-4e86-8f07-e4ac5469c159
quality_metric_fastqc in review by lab 1f30ebe0-4e50-4599-b3da-d313868a9980
quality_metric_fastqc in review by lab 1ef636af-7830-4a85-afb8-1062aa136116
quality_metric_fastqc in review by lab 13a3afc2-e3c2-4af4-88fb-58d96d8a5686
quality_metric_fastqc in review by lab 45f9f32b-fb0b-4449-88b9-1a2771eb40ce
quality_metric_fastqc in review by lab 6071137f-3a0e-42c4-9dad-7f3ab445e4aa
quality_metric_fastqc in review by lab bd098a73-bccc-4cd1-8a87-81344557bc2d
quality_metric_fastqc in review by lab 421f776c-e515-4217-81f7-18ca28b527a3
quality_metric_fastqc in review by lab 0a9e5ad5-20d6-4c0d-87dc-937c43c4991f
quality_metric_fastqc in review by lab 3f778ec7-e5ea-44ed-9686-cd2e53e5bb71
quality_metric_fastqc in review by lab d4d84ff7-43bc-4419-94a3-fac27e86103d
quality_metric_fastqc in review by lab 20fbfa5a-209e-4ce6-bb6c-02f7b0e4237d
quality_metr